# **Graph Link Prediction**

# 1. Preprocessing

## 1.1. Libraries

In [1]:
import torch
import os
import torch.nn as nn
import dgl
from dgl.nn import GraphConv
import random
import numpy as np
import networkx as nx
from karateclub import Node2Vec
from dgl.data import CoraGraphDataset
from sklearn.metrics import roc_auc_score
import torch.optim as optim
import torch.nn.functional as F
from sklearn.model_selection import train_test_split

os.environ["DGLBACKEND"] = "pytorch"


## 1.2. Load dataset

Φορτώνουμε το **CoraGraphDataset** dataset απο την βιβλιοθήκη **DGL**, το μετατρέπουμε σε **undirected  graph SimpleGraph** και παίρνουμε το μεγαλύτερο **connected component** διότι το γράφημα είναι **disconnected**

In [ ]:
dataset = CoraGraphDataset()
g_dgl = dataset[0]

# Convert to undirected NetworkX graph
nx_g = g_dgl.to_networkx()
G_und = nx_g.to_undirected()
G_simple = nx.Graph(G_und)

# Get largest connected component
components = list(nx.connected_components(G_simple))
giant = max(components, key=len)
print("Original nodes:", g_dgl.num_nodes(), "Original edges:", g_dgl.num_edges())
print("Number of components (undirected):", len(components))
print("Largest component size:", len(giant))

G = G_simple.subgraph(giant).copy()

print("Nodes in largest component (undirected NetworkX):", G.number_of_nodes())
print("Edges in largest component (undirected NetworkX):", G.number_of_edges())

## 1.3. G_train preparation

Παίρνουμε προτεινόμενες ακμές από το γραφήμα **G** κάνοντας ελεγχος για **self-loops**, **multiple edges** και εάν έχει **bridges** μεταξύ ακμών

In [4]:
# Check for self-loops and multiple edges for candidate edges
edges = set()

for u,v in G.edges():
    if u != v:
        edges.add((min(u, v), max(u, v)))

# Get G's bridges
bridges = set()
for u,v in nx.bridges(G):
    edge = (min(u, v), max(u, v))
    bridges.add(edge)

# Check if candidate edges are bridges
candidate_edges = []

for e in edges:
    if e not in bridges:
        candidate_edges.append(e)

candidate_num_edges = len(candidate_edges)
percentage_to_remove = 0.10

num_to_remove = int(candidate_num_edges * percentage_to_remove)

edges_removed = []

G_train = G.copy()

candidate_edges_shuffled = candidate_edges.copy()
random.shuffle(candidate_edges_shuffled)

Στην συνέχεια αφαιρούμε 10% των προτεινόμενων ακμών στο **G_Train** και μετα την αφαίρεση ακμών ελέγχουμε εάν το υπολειπόμενο γράφημα **G_Train** παραμένει **connected**

In [5]:
for u, v in candidate_edges_shuffled:
    if len(edges_removed) == num_to_remove:
        break

    # Check degree first
    if G_train.degree(u) <= 1 or G_train.degree(v) <= 1:
        continue

    # Check if edge is currently a bridge
    if (u, v) in nx.bridges(G_train) or (v, u) in nx.bridges(G_train):
        continue

    G_train.remove_edge(u, v)
    edges_removed.append((u, v))

assert nx.is_connected(G_train), "Training graph is disconnected"

# Test edges
test_positive_edges = edges_removed
nodes = list(G_train.nodes())

## 1.4. Negative Sampling

Παίρνουμε και τις **αρνητικές ακμές**(ακμές που δεν υπάρχουν μεταξύ κόμβων), έτσι ώστε να βάλουμε **labels** για θετίκες[1] και αρνητικές ακμες[0] που θα χρειαστούν για το **training** του μοντελού μας πιο μετά

In [6]:
test_negative_edges = []

for _ in range(num_to_remove):
    u,v = random.sample(nodes, 2)

    if not G_train.has_edge(u,v):
        test_negative_edges.append((u,v))

all_test_edges = test_positive_edges + test_negative_edges

test_labels_heu = [1]*len(test_positive_edges) + [0]*len(test_negative_edges)

# 2. Heuristic methods

Τρέχουμε 3 ευριστικές μεθόδους **Common Neighbors**, **Adamic–Adar Index** και **Jaccard Coefficient** πάνω στο υπολειπόμενο γράφημα **G_train** χρησιμοποιώντας τις **labeled** συνολικές ακμές

In [ ]:
common_neighbors_scores = []

for u,v in all_test_edges:
    common_neighbors = list(nx.common_neighbors(G_train, u,v))
    score = len(common_neighbors)
    common_neighbors_scores.append(score)

aa_index_scores = []

aa_index = list(nx.adamic_adar_index(G_train, all_test_edges))
for u,v,score in aa_index:
    aa_index_scores.append(score)

jaccard_scores = []

jaccard = list(nx.jaccard_coefficient(G_train, all_test_edges))
for u,v,score in jaccard:
    jaccard_scores.append(score)

common_neighboors_auc = roc_auc_score(test_labels_heu, common_neighbors_scores)
aa_index_auc = roc_auc_score(test_labels_heu, aa_index_scores)
jaccard_auc = roc_auc_score(test_labels_heu, jaccard_scores)

print("Common Neighbors AUC:", common_neighboors_auc)
print("Adamic-Adar AUC:", aa_index_auc)
print("Jaccard AUC:", jaccard_auc)


# 3. Feature extraction, Shallow embeddings (Node2Vec) and MLP setup and training/evaluation

In [ ]:
# Shallow embeddings
model = Node2Vec(dimensions=128,walk_length=80,walk_number=10,p=1,q=1)

**Reindexing** διότι o **Node2Vec** απαιτεί από τους κόμβους του γραφήματος **G** να είναι αριθμημένοι σωστά απο **0...Ν-1**

In [ ]:
# Reindexing
mapping = {}

for new_index, old_node in enumerate(G.nodes()):
    mapping[old_node] = new_index

G_reindexed = nx.relabel_nodes(G, mapping)

model.fit(G_reindexed)
embeddings = model.get_embedding()
print("Embeddings shape:", embeddings.shape)
print("First node embedding (first 5 values):", embeddings[0][:5])
print("Embedding mean/std:", embeddings.mean(), embeddings.std())


Υπολόγιζουμε το **Hadamard product** **$h_{uv} = z_u \odot z_v$** για κάθε ζευγος **$(u, v)$** **positive** και **negative** ακμών, ετσί ωστε να φτιαξούμε **embeddings** για τις ακμές

In [11]:
# Hadamard product for test edges
positive_edge_embeddings = []
negative_edge_embeddings = []

for u,v in  test_positive_edges:
    u_i = mapping[u]
    v_i = mapping[v]
    positive_edge_product = embeddings[u_i] * embeddings[v_i]
    positive_edge_embeddings.append(positive_edge_product)

for u,v in test_negative_edges:
    u_i = mapping[u]
    v_i = mapping[v]
    negative_edge_product = embeddings[u_i] * embeddings[v_i]
    negative_edge_embeddings.append(negative_edge_product)



## 3.1. Test Set setup

Δημιουργούμε για το **test** σύνολο **feature embedings** **(X_train)** και **labels** **(Y_train)** για το τελικο **evaluation**

In [ ]:
# Test set
X_test = np.array(positive_edge_embeddings + negative_edge_embeddings, dtype=np.float32)
X_test = torch.tensor(X_test)
print("X_test: ",X_test)

test_labels = [1]*len(positive_edge_embeddings) + [0]*len(negative_edge_embeddings)
Y_test = np.array(test_labels, dtype=np.float32)
Y_test = torch.tensor(Y_test).unsqueeze(1)
print("Y_test : ",Y_test)

## 3.2. Training edges and Train/Evaluation split sets

Χρησιμοποιούμε το γράφημα **G_train** για να παρουμε θετικές και αρνητικές ακμές

In [13]:
# Training edges
train_positive_edges = list(G_train.edges())
train_nodes = list(G_train.nodes())
train_negative_edges = []

while len(train_negative_edges) < len(train_positive_edges):
    u,v = random.sample(train_nodes, 2)
    if not G_train.has_edge(u,v):
        train_negative_edges.append((u, v))


Χωρίζουμε τα **train_positive_edges** και **train_negative_edges** σε **80% train** και **20% validation**

In [14]:
# Evaluation and Train split sets

train_possitive, val_possitive = train_test_split(train_positive_edges,
                                        test_size = 0.2,
                                        random_state = 42,
                                        shuffle = True)

train_negative, val_negative = train_test_split(train_negative_edges,
                                        test_size = 0.2,
                                        random_state = 42,
                                        shuffle = True)

Δημιουργούμε για το **train** σύνολο **feature embedings** **(X_train)** και **labels** **(Y_train)** για να εκπαιδεύσουμε το μοντέλο μας

In [ ]:
X_train = []
Y_train = []

for u,v in train_possitive:
    u_i = mapping[u]
    v_i = mapping[v]
    edge_embedding = embeddings[u_i] * embeddings[v_i]
    X_train.append(edge_embedding)
    Y_train.append(1)

for u,v in train_negative:
    u_i = mapping[u]
    v_i = mapping[v]
    edge_embedding = embeddings[u_i] * embeddings[v_i]
    X_train.append(edge_embedding)
    Y_train.append(0)

X_train = torch.tensor(np.array(X_train), dtype=torch.float32)
Y_train = torch.tensor(Y_train, dtype=torch.float32).unsqueeze(1)

# Shuffle
perm = torch.randperm(X_train.size(0))

X_train = X_train[perm]
Y_train = Y_train[perm]

print("X_train shape:", X_train.shape)
print("Y_train shape:", Y_train.shape)
print("First 5 labels:", Y_train[:5].T)
print("Positive/Negative ratio:",Y_train.sum().item(), "/", len(Y_train))

Δημιουργούμε για το **validation** σύνολο **feature embedings** **(X_val)** και **labels** **(Y_val)** για το συνεχές **monitoring** των αποτελεσμάτων

In [ ]:
X_val = []
Y_val = []

for u, v in val_possitive:
    u_i = mapping[u]
    v_i = mapping[v]
    edge_embedding = embeddings[u_i] * embeddings[v_i]
    X_val.append(edge_embedding)
    Y_val.append(1)

for u, v in val_negative:
    u_i = mapping[u]
    v_i = mapping[v]
    edge_embedding = embeddings[u_i] * embeddings[v_i]
    X_val.append(edge_embedding)
    Y_val.append(0)

X_val = torch.tensor(np.array(X_val), dtype=torch.float32)
Y_val = torch.tensor(Y_val, dtype=torch.float32).unsqueeze(1)

# Shuffle
perm = torch.randperm(X_val.size(0))

X_val = X_val[perm]
Y_val = Y_val[perm]

print("X_val shape:", X_val.shape)
print("Y_val shape:", Y_val.shape)
print("First 5 labels:", Y_val[:5].T)
print("Positive/Negative ratio:", Y_val.sum().item(), "/", len(Y_val))

# 3.3) Simple MLP setup

To μοντέλο MLP εχει 3 Linear Layers, για **criterion** χρησιμοποιούμε των **BCEWithLogitsLoss** της βιβλιοθήκης **torch.nn** και για **optimizer** τον **Adam** της βιβλιοθήκης **torch.optim**


In [17]:
class MLP(nn.Module):
    def __init__(self,embedding_dimension):
        super(MLP,self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(embedding_dimension, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64,1)
        )

    def forward(self,x):
        return self.linear_relu_stack(x)

# Setup model
embedding_dimension = embeddings.shape[1]
model_mlp = MLP(embedding_dimension=embedding_dimension)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model_mlp.parameters(), lr = 0.001)

# 3.4) Train MLP model

Περνάμε στις παραμετρους τα **feature embedings** **(X_train)** και **labels** **(Y_train)** του συνολου **validation** και **train** τον **optimizer** και **criterion** και των αριθμο passthrough στο **training set** **(epochs)**

In [ ]:
def train(model, X_train, Y_train, X_val, Y_val, criterion, optimizer, epochs=100):
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()

        logits = model(X_train)
        loss = criterion(logits, Y_train)
        loss.backward()
        optimizer.step()

        model.eval()
        with torch.no_grad():
            val_logits = model(X_val)
            val_probs = torch.sigmoid(val_logits)
            val_auc = roc_auc_score(Y_val.cpu().numpy(), val_probs.cpu().numpy())

        print(f"Epoch {epoch:03d} | Loss: {loss.item():.4f} | Val AUC: {val_auc:.4f}")

# Train model
train(model = model_mlp,
      X_train = X_train,
      Y_train = Y_train,
      X_val = X_val,
      Y_val = Y_val,
      criterion = criterion,
      optimizer = optimizer,
      epochs = 100
)

# 3.5) Evaluate MLP model

Τέλος βγαζουμε το **AUC** για το **test set** και **validation set**

In [ ]:
def evaluate(model, X, Y):
    model.eval()
    with torch.no_grad():
        logits = model(X)
        probs = torch.sigmoid(logits)
        auc = roc_auc_score(Y.cpu().numpy(), probs.cpu().numpy())
    return auc

# Final evaluation
val_auc = evaluate(model_mlp, X_val, Y_val)
test_auc = evaluate(model_mlp, X_test, Y_test)

print("Final Val AUC:", val_auc)
print("Final Test AUC:", test_auc)

# 4) GNN setup and training/evaluation

## 4.1) Convert training graph from NetworkX to DGL 

Τo **GNN** χρειάζετε το γράφημα να είναι σε **DGL** object

In [20]:
g_train_dgl = dgl.from_networkx(G_train)
g_train_dgl = dgl.add_self_loop(g_train_dgl)

node_list = list(G_train.nodes())
g_train_dgl.ndata["feat"] = g_dgl.ndata["feat"][torch.tensor(node_list)]


## 4.2) Test/Val split sets

Χωρίζουμε τα **test_positive_edges** και **test_negative_edges** σε **50% test** και **50% validation**

In [29]:
val_positive_edges, test_positive_edges = train_test_split(
    test_positive_edges,
    test_size=0.5,
    random_state=42
)

val_negative_edges, test_negative_edges = train_test_split(
    test_negative_edges,
    test_size=0.5,
    random_state=42
)

# 4.3) GCN encoder setup

Ο **GCN encoder** εχει 2 Layers με σκοπο να φτιάξει **embeddings** κόμβων


In [30]:
class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, out_feats):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, out_feats)
        
    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g,h)
        return h

# Setting up encoder (GCN)
in_feats = g_train_dgl.ndata["feat"].shape[1]
h_feats = 64
out_feats = 64
encoder = GCN(in_feats=in_feats, h_feats=h_feats, out_feats=out_feats)


# 4.4) Setup dot product decoder

Ο **dot product** χρησιμοποιήτε για **link prediction**, πέρνει δύο **embeddings** κόμβων και βγάζει ενα **score** για κάθε ακμή

In [31]:
def dot_product(z, edges):
    u = edges[:,0] 
    v = edges[:,1]
    y_hat = (z[u] * z[v]).sum(dim=1)
    return y_hat

# 4.5) Reindex test/val edges

Κάνουμε **reindex** τα **test edges** και **val edges** του **NetworkX training graph (G_train)** για να ταιριάξουν στο **DGL training graph (g_train_dgl)** έτσι ώστε να τα έχουμε έτοιμα για **evaluation** μετά

In [32]:
# Reindex test edges
new_mapping = {}
positive_mapped_test_edges = []
negative_mapped_test_edges = []

for new_index, old_node in enumerate(G_train.nodes()):
    new_mapping[old_node] = new_index

for u, v in test_positive_edges:
    if u in new_mapping and v in new_mapping:
        u_i = new_mapping[u]
        v_i = new_mapping[v]
        positive_mapped_test_edges.append((u_i, v_i))

for u, v in test_negative_edges:
    if u in new_mapping and v in new_mapping:
        u_i = new_mapping[u]
        v_i = new_mapping[v]
        negative_mapped_test_edges.append((u_i, v_i))

# Reindex val edges
positive_mapped_val_edges = []
negative_mapped_val_edges = []

for u, v in val_positive_edges:
    if u in new_mapping and v in new_mapping:
        u_i = new_mapping[u]
        v_i = new_mapping[v]
        positive_mapped_val_edges.append((u_i, v_i))

for u, v in val_negative_edges:
    if u in new_mapping and v in new_mapping:
        u_i = new_mapping[u]
        v_i = new_mapping[v]
        negative_mapped_val_edges.append((u_i, v_i))

# 4.6) Train GNN model

Περνάμε στις παραμετρους το **encoder model (GCN)**, το **training graph**, τα **features**, τον **optimizer (Adam)**, τον **decoder (dot_product)**, των αριθμό **epoch** για το **training loop** και τελος ποσα αρνητικα δείγματα θελουμε ανα θετική ακμή, χρησιμοποιούμε **Binary cross entroopy with logits** για **loss function**

In [ ]:
def evaluate_gnn(model, decoder, g, node_feats, pos_edges, neg_edges):
    model.eval()

    with torch.no_grad():
        z = model(g, node_feats)
        positive_edges = torch.tensor(pos_edges)
        negative_edges = torch.tensor(neg_edges)

        positive_score = decoder(z, positive_edges)
        negative_score = decoder(z, negative_edges)

        scores = torch.cat([positive_score, negative_score])

        positive_label = torch.ones(len(positive_score)) 
        negative_label = torch.zeros(len(negative_score))

        labels = torch.cat([positive_label, negative_label])


        auc = roc_auc_score(labels, scores)

    return auc

def train_gnn(model, g_train, node_feats, val_pos, val_neg, optimizer, decoder, epochs=100, neg_samples=1):

    for epoch in range(epochs):
        model.train()
        # Encoding edges
        z = model(g_train, node_feats)

        # Getting possitive edges
        u, v = g_train.edges()
        positive_edges = torch.stack([u, v], dim=1)
        positive_score = decoder(z, positive_edges)
        positive_label = torch.ones_like(positive_score)

        # Negative sampling
        num_negative = u.shape[0] * neg_samples
        negative_u = torch.randint(0, g_train.num_nodes(), (num_negative,))
        negative_v = torch.randint(0, g_train.num_nodes(), (num_negative,))
        negative_edges = torch.stack([negative_u, negative_v], dim=1)
        negative_score = decoder(z, negative_edges)
        negative_label = torch.zeros_like(negative_score)

        
        scores = torch.cat([positive_score, negative_score])
        labels = torch.cat([positive_label, negative_label])

        loss = F.binary_cross_entropy_with_logits(scores, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        with torch.no_grad():
             val_auc = evaluate_gnn(
                    model=model,
                    decoder=decoder,
                    g=g_train,
                    node_feats=node_feats,
                    pos_edges=val_pos,
                    neg_edges=val_neg
                )

        print(f"Epoch {epoch:03d} | Loss: {loss.item():.4f} | Val AUC: {val_auc:.4f}")

# Setting up optimizer

optimizer = torch.optim.Adam(encoder.parameters(), lr=0.01)

# Train model

train_gnn(
    model = encoder,
    g_train = g_train_dgl,
    node_feats = g_train_dgl.ndata["feat"],
    val_pos = positive_mapped_val_edges,
    val_neg = negative_mapped_val_edges,
    optimizer = optimizer,
    decoder = dot_product,
    epochs = 100,
    neg_samples = 1
)


# 4.7) Evaluate GNN model

Τέλος βγαζουμε το **AUC** για τις **test** θετικές και αρνητικές ακμές

In [ ]:
# Final Evaluation 
test_auc = evaluate_gnn(
    model = encoder,
    decoder = dot_product,
    g = g_train_dgl,
    node_feats = g_train_dgl.ndata["feat"],
    pos_edges = positive_mapped_test_edges,
    neg_edges = negative_mapped_test_edges
)

val_auc = evaluate_gnn(
    model=encoder,
    decoder=dot_product,
    g=g_train_dgl,
    node_feats=g_train_dgl.ndata["feat"],
    pos_edges=positive_mapped_val_edges,
    neg_edges=negative_mapped_val_edges
)

print("Final Val AUC:", val_auc)
print("Final Test AUC:", test_auc)